In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pandas import Series, DataFrame
from selenium.webdriver.common.keys import Keys
import time
import re

In [10]:
def sort_test():
       
    try:
        driver.find_element_by_link_text('More')
        is_more = True
    except NoSuchElementException:
        is_more = False   

    if is_more:
        success = 0
        while success < 2:
            try:
                #attractions_card = driver.find_elements_by_class_name('attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd')              
                more_button = driver.find_element_by_link_text('More')
                driver.execute_script("arguments[0].click();", more_button)
                success = 2
                print('successed to push more!')
            except NoSuchElementException:
                more_button = driver.find_element_by_link_text('More')
                more_button.send_keys(Keys.ENTER)
                success += 1
    else:
        print("No More!")
        
    #more부분 클릭
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    attraction_header = soup.find(class_ = 'attractions-attraction-review-atf-AttractionReviewATFLayout__atf_left_column--1ogzd')
    sorting = attraction_header.find(class_ = "attractions-attraction-review-header-AttractionLinks__detail--2-xvX")
    
# 원하는 정보가 담겨있는 전체 페이지 선택
    try:
        location = attraction_header.find('span', class_='attractions-attraction-review-atf-overview-card-Address__address--1QyPi').text
    except:
        location = float('nan')
    try :
        small_sorting = sorting.find_all(class_ = 'attractions-attraction-review-header-AttractionLinks__dotted_link--Pt2MP')[1].text,
        small_sorting = small_sorting[0]
    except:
        small_sorting = float('nan')
    try :
        big_sorting = sorting.find(class_ = 'attractions-attraction-review-header-AttractionLinks__dotted_link--Pt2MP').text
    except:
        big_sorting = float('nan')
    sort = [attraction_header.find('h1', class_='ui_header h1').text,
        big_sorting,
        small_sorting,
        location
    ]

    return sort

In [8]:
def load_reviews(link):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    time.sleep(5)
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
                
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = sort_test()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    
    while len(driver.window_handles) == 2:
        driver.close();
        time.sleep(3)

    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    
    print(result)
    
    return result
    

In [4]:
def load_page(start, end):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print("I am in the load_page")
    if driver.current_url == 'https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html':
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    else:
        main_page = driver.find_element_by_class_name('scrollAdMain')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')
        is_first = False
    print("I found the elements!")

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            div_class = attraction_element.find_element_by_class_name('tracking_attraction_title')
            link = div_class.find_element_by_tag_name('a')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))   
    print("I found the attraction_names")
    
    columns = ['name','big sortig','small sorting','address']
    attraction_list = []
    for attraction_name, link in attraction_names:
        if len(attraction_list) < end:
            attraction_list.append(load_reviews(link))
            if len(attraction_list) % 5 == 0:
                data_to_csv(DataFrame(attraction_list, columns = columns), start, end)
                print("저장했어요!" + str(start)+'-'+ str(end))
                
        else:
            data_to_csv(attraction_dict)
            print("저장했어요!")
            break

In [22]:
def dict_n(start, end):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
    #처음 한 번만 see more을 누른다.
    is_first_page = True
    try:
        time.sleep(2)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)        
        is_first_page = True
        print("it is first page!")
    except NoSuchElementException:
        print('continue!')
        is_first_page = False
    except:
        print("exception")
           
    time.sleep(5)
    keep_going = True
    while keep_going:
        
        # 현재 페이지 번호 알아내기

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print("I made a soup")
        if is_first_page:
            selected_page = 1
            page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
            if (start // 30) > 0:
                assert (start // 30) > 0
                if (start // 30) > 6:
                    target_num = 6
                else:
                    target_num = (start // 30) + 1
                target_page = page_numbers.find_element_by_link_text(str(target_num))
                time.sleep(5)
                driver.execute_script("arguments[0].click();", target_page)
                time.sleep(3)
                selected_page = target_num
                page_numbers = driver.find_element_by_class_name('pageNumbers')
        else:
            selected_page = int(soup.find("div", class_='pageNumbers').find(class_ = 'current').text)
            page_numbers = driver.find_element_by_class_name('pageNumbers')
        print("I find the current page")
        
        # 시작점이 뒷 페이지면 지나간다
        if start <= selected_page * 30:
            if start < 0:
                start = abs(start)
                print("start is small than 0")
            
            strat_in_page = (start) - ((selected_page-1)*30)

            if strat_in_page < 0:
                strat_in_page = 0
                
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            load_page(strat_in_page, end_in_page)
            
            if is_first_page:
                is_first_page = False
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
        # 다음 페이지로 넘어가기
        time.sleep(5)
        next_num = str(selected_page + 1)
        next_page = page_numbers.find_element_by_link_text(next_num)
        driver.execute_script("arguments[0].click();", next_page)

In [6]:
def data_to_csv(attraction_sort, start, end):
    '''
    입력 : 딕셔너리 데이터
    출력 : csv파일 형태로 원시 자료 폴더에 출력.
    '''
    git_address = 'C:/Users/taehee/Documents/GitHub/'
    attraction_sort.to_csv(git_address + str(start)+'-'+ str(end) + '.csv', mode = 'w')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("lang=en-CA")
options.add_argument('window-size=1920x1080')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = r'C:/Users/taehee/Downloads/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(driver_address, chrome_options=options)
wait = WebDriverWait(driver, timeout=2)
driver.implicitly_wait(5)
# tripadvisor에 접근한다.
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')
#driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-oa30-Seoul.html')
#원래 있던 리스트 창을 저장해둔다
original_window = driver.current_window_handle

dict_n(start = 0,end = 180)

C:\Users\taehee\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
